In [2]:
import pandas as pd
import geopandas as gpd
import numpy as np
import folium

# ------------ paths ------------
FULL_SHP     = "2020_Census_Tracts_in_Boston.shp"            # full attrs
REDUCED_SHP  = "2020_Census_Tracts_in_Boston_Reduced.shp"    # geometry-only
CSV_FILE     = "tracts_median_income.csv"
GEOJSON_OUT  = "boston_income.geojson"
HTML_OUT     = "boston_income_map.html"

# ------------ 1) full tracts with GEOID ------------
full = gpd.read_file(FULL_SHP)

# standardize to 'geoid20'
geoid_col = next((c for c in ["geoid20", "GEOID20", "geoid", "GEOID"] if c in full.columns), None)
if geoid_col is None:
    if all(c in full.columns for c in ["statefp20", "countyfp20", "tractce20"]):
        full["geoid20"] = (
            full["statefp20"].astype(str).str.zfill(2) +
            full["countyfp20"].astype(str).str.zfill(3) +
            full["tractce20"].astype(str).str.zfill(6)
        )
        geoid_col = "geoid20"
    else:
        raise ValueError("Full tracts file lacks GEOID fields.")

if geoid_col != "geoid20":
    full = full.rename(columns={geoid_col: "geoid20"})

full = full[["geoid20", "geometry"]].copy()

# ------------ 2) reduced geometry + spatial join ------------
reduced = gpd.read_file(REDUCED_SHP)
if reduced.crs is None:
    reduced = reduced.set_crs(full.crs)
elif reduced.crs != full.crs:
    reduced = reduced.to_crs(full.crs)

reduced_cent = reduced.copy()
reduced_cent["geometry"] = reduced_cent.geometry.centroid
reduced_with_geoid = gpd.sjoin_nearest(
    reduced_cent[["geometry"]],
    full[["geoid20", "geometry"]],
    how="left"
)
reduced = reduced.join(reduced_with_geoid["geoid20"])
if reduced["geoid20"].isna().any():
    print(f"⚠️ {int(reduced['geoid20'].isna().sum())} polygons didn’t get a GEOID.")

# ------------ 3) income CSV merge ------------
inc = pd.read_csv(CSV_FILE)
inc["state"] = inc["state"].astype(str).str.zfill(2)
inc["county"] = inc["county"].astype(str).str.zfill(3)
inc["tract"] = inc["tract"].astype(str).str.zfill(6)
inc["geoid20"] = inc["state"] + inc["county"] + inc["tract"]
inc["median_income"] = pd.to_numeric(inc["median_income"], errors="coerce")
inc.loc[inc["median_income"] <= 0, "median_income"] = pd.NA

merged = reduced.merge(inc[["geoid20", "median_income"]], on="geoid20", how="left")
merged["income_str"] = merged["median_income"].apply(
    lambda x: f"${x:,.0f}" if pd.notna(x) else "N/A"
)

# ------------ 4) export GeoJSON ------------
merged.to_crs(4326).to_file(GEOJSON_OUT, driver="GeoJSON")
print(f"✅ GeoJSON exported: {GEOJSON_OUT}")

# ------------ 5) build bins ------------
valid = merged["median_income"].dropna()
if len(valid) >= 3:
    q = np.quantile(valid, [0, .15, .3, .45, .6, .75, .9, 1])
    q = np.round(q / 1000) * 1000
    bins = np.unique(q.astype(int))
    lo = int(np.floor(valid.min() / 1000) * 1000)
    hi = int(np.ceil(valid.max() / 1000) * 1000)
    bins[0] = min(bins[0], lo)
    bins[-1] = max(bins[-1], hi)
    if bins.size < 3:
        bins = np.linspace(lo, hi, 6).astype(int)
else:
    bins = None

print(
    f"Data summary — min: {valid.min() if len(valid) else 'NA'}, "
    f"max: {valid.max() if len(valid) else 'NA'}, "
    f"n_valid: {len(valid)}, bins: {bins.tolist() if isinstance(bins, np.ndarray) else 'auto'}"
)

# ------------ 6) folium map ------------
m = folium.Map(location=[42.36, -71.06], zoom_start=11, tiles=None)
folium.TileLayer("cartodbpositron", control=False).add_to(m)

choropleth_kwargs = dict(
    geo_data=merged.to_crs(4326).to_json(),
    data=merged,
    columns=["geoid20", "median_income"],
    key_on="feature.properties.geoid20",
    fill_color="YlGnBu",
    fill_opacity=0.75,
    line_opacity=0.6,
    nan_fill_color="#f6f6f6",
    name="Median income (choropleth)",
    legend_name="Median Household Income (USD)"
)

if bins is not None:
    folium.Choropleth(bins=bins.tolist(), **choropleth_kwargs).add_to(m)
else:
    folium.Choropleth(scheme="Quantiles", k=5, **choropleth_kwargs).add_to(m)

# ------------ 7) Add hover tooltip layer ------------
folium.GeoJson(
    merged.to_crs(4326),
    name="GEOID & Income",
    style_function=lambda x: {"color": "#444", "weight": 0.6, "fillOpacity": 0},
    tooltip=folium.GeoJsonTooltip(
        fields=["geoid20", "income_str"],
        aliases=["GEOID", "Median Income"],
        localize=True,
        sticky=True,
        labels=True,
    )
).add_to(m)

folium.LayerControl(collapsed=True).add_to(m)
m.save(HTML_OUT)
print(f"✅ Map saved: {HTML_OUT}")

✅ GeoJSON exported: boston_income.geojson
Data summary — min: 33229.0, max: 151466.0, n_valid: 21, bins: [33000, 60000, 79000, 82000, 89000, 93000, 117000, 152000]
✅ Map saved: boston_income_map.html
